**Building a dataset to analyze the correlation between GTO Fixed Income trading
with that of US Treasuries and the volatility index (MOVE Index)**

### **Run our regressions over a 5yr period for Daily data with the MOVE Index**

In this analysis, we will not have to resample the data in the same series since all data is now daily

In [ ]:
pip install pmdarima

In [ ]:
#load libraries

import pandas as pd
import numpy as np
import datetime as dt 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)

# Load specific forecasting tools
from statsmodels.tsa.arima_model import ARMA,ARMAResults,ARIMA,ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS Plots
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

#Writing data to excel to share with Exec team
# !pip install XlsxWriter
# !pip install openpyxl
# import openpyxl
# from openpyxl import Workbook

In [ ]:
#Load our datasets
DATA_PATH = "/content/drive/My Drive/Projects/FixedIncome_TA"
fi_data = open(DATA_PATH+'/DailyTradeCounts_2016_2020.xlsx','rb')
move1_data = open(DATA_PATH+'/MOVE Index 16 18.xlsx','rb')
move2_data = open(DATA_PATH+'/MOVE Index Data.xlsx','rb')
pd_data = open(DATA_PATH+'/NY Fed Primary Dealer Trade Volumes.xlsx','rb')
# df = pd.read_csv(url1,index_col='Date',parse_dates=True )

In [ ]:
#Load our datasets

FT_pg1 = pd.read_excel(fi_data, sheet_name="20160101_20181130")
FT_pg2 = pd.read_excel(fi_data, sheet_name="20181201_20200730")
FT_volumes = pd.concat([FT_pg1,FT_pg2])  #combine the data on top,not a new column

move_daily1 = pd.read_excel(move1_data, sheet_name="Sheet1", 
                           header=1,skiprows=5)

move_daily2 = pd.read_excel(move2_data, sheet_name="2yr Daily", 
                           header=1,skiprows=4)

dealer_volumes_df = pd.read_excel(pd_data, sheet_name="Sheet1")


In [ ]:
FT_volumes.head()

In [ ]:
from datetime import datetime
FT_volumes['Date'] = pd.to_datetime(FT_volumes['Date'].astype(str), format='%Y-%m-%d')
FT_volumes.rename(columns={'Non Premium': 'FI Trades'}, inplace=True)

weekly_volumes = FT_volumes.groupby("Client").resample('W-Wed',
                                                      label='right', 
                                                      closed = 'right',
                                                      on='Date').sum().reset_index().sort_values(by='Date')

weekly_volumes = weekly_volumes.groupby(['Date'])['FI Trades'].agg('sum')
weekly_df = pd.DataFrame(weekly_volumes)
weekly_df.head()

In [ ]:
weekly_df = weekly_df[:-1]  #remove an august, 2020 print
weekly_df.tail()

## Plot out the Trade Volume time series and ETS decomposition

In [ ]:
title = 'Weekly Fixed Income Trading Volume'

ylabel='Millions of Trades'
xlabel='Date'

ax = weekly_df.plot(figsize=(12,6), title=title)
low = weekly_df.min()
high = weekly_df.max()
ticks = np.linspace(low,high,15).astype(int)
ax.set(xlabel=xlabel, ylabel=ylabel)
ax.set_yticks(ticks);


In [ ]:
ets = seasonal_decompose(weekly_df,model='add')
ets.plot();

In [ ]:
#seems to have a seasonal component
#reviewing the first couple seasons
weekly_df['FI Trades'][:152].plot();
#during end of dec, early jan dropoff, as expected for holiday

## Cleaning up the data from the MOVE Index

In [ ]:

move_daily1.rename(columns={'Date':'Day','Last Px': 'Date', 'Unnamed: 2':'Last Px'}, inplace=True)

#stitch the files together
move_daily = pd.concat([move_daily2,move_daily1],ignore_index=True)
move_daily.drop(['Day','Unnamed: 3'], axis=1, inplace=True)
move_daily.dropna(inplace=True)
move_daily.reset_index(drop=True, inplace=True)

move_index= move_daily[['Date','Last Px']]
move_index.rename(columns={'Last Px': 'MOVE Index'}, inplace=True)
move_index['MOVE Index'] = pd.to_numeric(move_index['MOVE Index'], errors='coerce')
move_index['Date'] = pd.to_datetime(move_index['Date'], errors='coerce')
move_index["Date"] = pd.to_datetime(move_index["Date"], format = '%Y%m').dt.to_period('d')
move_index = move_index.dropna()
move_index = move_index.reset_index(drop=True)


In [ ]:
#fix the starting point of the move index to align weekly time series
date = '2016-01-04'
move_index = move_index[move_index['Date']>(date)]

from datetime import datetime
move_index['Date'] = pd.to_datetime(move_index['Date'].astype(str), format='%Y-%m-%d')

move_index.set_index('Date', inplace=True)

#align our dates to fall on same day as others 
move_weekly = move_index['MOVE Index'].resample('W-WED', origin='start').mean()

move_weekly.head()


In [ ]:
move_df = pd.DataFrame(move_weekly)
move_df.head()

In [ ]:
move_df.tail()

## Building the Dealer dataframe

In [ ]:
dealer_volumes_df.dropna(axis=1, inplace=True)
# dealer_volumes_df.head()
dealer_volumes_df = dealer_volumes_df.pivot(index="As Of Date",columns="Timeseries", values='Value (millions)') \
       .reset_index().rename_axis(None, axis=1)
dealer_volumes_df.rename(columns={'As Of Date': 'Date',
                                  'PDTRGSC-L2':'< 2yr duration',
                                  'PDTRGSC-G2L3':'2-3yr duration',
                                  'PDTRGSC-G3L6':'3-6yr duration',
                                  'PDTRGSC-G6L7':'6-7yr duration',
                                  'PDTRGSC-G7L11':'7-11yr duration',
                                  'PDTRGSC-G11':'> 11yr duration',
                                  }, 
                         inplace=True)

In [ ]:
dealer_volumes_df.head()

In [ ]:

columnsTitles = ['Date','< 2yr duration','2-3yr duration',
                 '3-6yr duration','6-7yr duration',
                 '7-11yr duration','> 11yr duration']
cols = ['< 2yr duration','2-3yr duration',
                 '3-6yr duration','6-7yr duration',
                 '7-11yr duration','> 11yr duration']
dealer_volumes_df = dealer_volumes_df.reindex(columns=columnsTitles)
dealer_volumes_df[cols] = dealer_volumes_df[cols].apply(pd.to_numeric, errors='coerce')

dealer_volumes_df['Date']= pd.to_datetime(dealer_volumes_df['Date'])
dealer_volumes_df[['< 2yr duration','2-3yr duration',
                 '3-6yr duration','6-7yr duration',
                 '7-11yr duration','> 11yr duration']].astype(np.float64)

dealer_volumes_df["All Treasury Volume"] = dealer_volumes_df[cols].sum(axis=1) 
# dealer_volumes_df['> 11yr series'] = pd.to_numeric(dealer_volumes_df['> 11yr series'])
print(dealer_volumes_df.dtypes)
print(dealer_volumes_df[dealer_volumes_df['All Treasury Volume'].isnull()].shape)
dealer_volumes_df.set_index('Date')
dealer_volumes_df.head(5)

In [ ]:
dealer_volumes_df.set_index('Date', inplace=True)

print(dealer_volumes_df.dtypes)
print(dealer_volumes_df[dealer_volumes_df['All Treasury Volume'].isnull()].shape)
dealer_volumes_df.head(5)

In [ ]:
weekly_df = weekly_df.join(dealer_volumes_df)

weekly_df.head()

In [ ]:
#Bring all dataframes together

weekly_df = weekly_df.join(move_df)
weekly_df.head()

In [ ]:
weekly_df.tail()

In [ ]:
weekly_df['MOVE Index'].ffill(axis=0, inplace=True)
weekly_df['MOVE Index'].tail()

The MOVE Index is missing data from July.  For now, we're fix that so we can run a balanced analysis but will look to complete that data going forward.

In [ ]:
# Plot two lines with different scales on the same plot

import matplotlib.ticker as ticker
formatter = ticker.StrMethodFormatter('{x:,.0f}')

fig = plt.figure(figsize=(12, 5))
line_weight = 3
alpha = .5
ax1 = fig.add_axes([0, 0, 1, 1])
ax2 = fig.add_axes()
# This is the magic that joins the x-axis
ax2 = ax1.twinx()
lns1 = ax1.plot(weekly_df['FI Trades'], color='blue', lw=line_weight, alpha=alpha, label='FI Trades Volume')
lns2 = ax2.plot(weekly_df['All Treasury Volume'], color='green', lw=line_weight, alpha=alpha, label='All Dealer Activity')
# Solution for having two legends
leg = lns1 + lns2
labs = [l.get_label() for l in leg]
ax1.legend(leg, labs, loc='best')
plt.title('Weekly Fixed Income Volume & Total Primary Dealer Activity in Notional Value', fontsize=20)
ax1.set(ylabel="FI Volume (mlns)")  
ax2.set(ylabel="Dealer Notional Value (mlns)")

ax2.yaxis.set_major_formatter(formatter);
plt.savefig('fivol_dealer.png',dpi=60, bbox_inches = "tight")
plt.show()

In [ ]:
# Plot two lines with different scales on the same plot
fig = plt.figure(figsize=(12, 5))
line_weight = 3
alpha = .5
ax1 = fig.add_axes([0, 0, 1, 1])
ax2 = fig.add_axes()
# This is the magic that joins the x-axis
ax2 = ax1.twinx()
lns1 = ax1.plot(weekly_df['FI Trades'], color='blue', lw=line_weight, alpha=alpha, label='FI Trades')
lns2 = ax2.plot(weekly_df['MOVE Index'], color='orange', lw=line_weight, alpha=alpha, label='MOVE Index')
# Solution for having two legends
leg = lns1 + lns2
labs = [l.get_label() for l in leg]
ax1.legend(leg, labs, loc='best')
plt.title('Weekly Fixed Income Trading Activity and MOVE Index', fontsize=20)
ax1.set(ylabel="FI Volume (mlns)")  
ax2.set(ylabel="MOVE Index Level")

plt.savefig('fivol_move.png',dpi=60, bbox_inches = "tight")
plt.show()

Checking data for stationarity

In [ ]:
# INCLUDED HERE IF YOU CHOOSE TO USE IT
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(weekly_df['FI Trades'], title='Weekly Fixed Income Volumes')

In [ ]:
adf_test(weekly_df['MOVE Index'], title='Weekly Fixed Income Volumes')

In [ ]:
adf_test(weekly_df['All Treasury Volume'], title='Weekly Fixed Income Volumes')
# weekly_df.columns

In [ ]:
weekly_df.head()

In [ ]:
#Looking at the ACF and PCF graphs to determine where the FI Trades series could become stationary.
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,5))

print('This plot indicates non-stationary data, as there are a large number of lags before ACF values drop off.')
title = 'Autocorrelation: FI Trades'
lags = 40
plot_acf(weekly_df['FI Trades'],title=title,lags=lags, ax=ax);

In [ ]:
from statsmodels.tsa.statespace.tools import diff

weekly_df['d1'] = diff(weekly_df['FI Trades'],k_diff=1)
weekly_df['d1'].plot(figsize=(12,5));

title='Autocorrelation: FI Trades First Differencing'
lags=40
plot_pacf(weekly_df['d1'].dropna(),title=title,lags=np.arange(lags));

We can visually see from the plot above that a First Difference lag can transform the Trading series in a stationary one for analyzing.  Another way to understand our dataset and more efficient is to run the auto_arima function.  This provides more concrete detail on how we should model this series.

In [ ]:
#Run ARIMA model to compare AIC score to SARIMA
auto_arima(weekly_df['FI Trades'],seasonal=False).summary()


In [ ]:
auto_arima(weekly_df['FI Trades'],seasonal=True,m=52).summary()

We can see a slight improvement to the AIC score when incorporating seasonality.  The AIC is meant to penalize a model given the added complexity and despite that, our SARIMA model still showed some improvement.  

In [ ]:
#Create a variable to analyze our observations for testing and prediction.  Let's forecast for 6mos.
nobs = 4 * 6 # weeks * mos
# Set four weeks for testing
train, test = weekly_df[0:-nobs], weekly_df[-nobs:]

In [ ]:
train.shape
test.shape


In [ ]:
	# SARIMAX(0, 1, 2)x(1, 0, [], 52)
model = SARIMAX(train['FI Trades'],order=(0,1,2),enforce_invertibility=False,seasonal_order=(1,0,0,52))
results = model.fit()
results.summary() 

In [ ]:
start=len(train)
end=len(train)+len(test)-1
predictions = results.predict(start=start, end=end, dynamic=False, typ='levels').rename('SARIMAX(0, 1, 2)x(1, 0, [], 52) Predictions')

In [ ]:
# Plot predictions against known values
title = 'Weekly Fixed Income Trading Volumes'
ylabel='Volume in mlns'
xlabel=''

ax = test['FI Trades'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);

As we can see above, the first couple months the model didn't track very well.  We failed to capture the spike in activity potentially indicating an exogenous feature contributing to the original data.

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test['FI Trades'], predictions)
print(f'SARIMAX(0, 1, 2)x(1, 0, [], 52)  RMSE Error: {error:11.10}')

In [ ]:
test_mean = test['FI Trades'].mean()
err_val = error/test_mean
print(f'Mean of the test set: {test_mean:11.8}')
print(f'Our relative error: {err_val:11.5}')


Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors

Let's try including MOVE Index and then the Primary Dealer data

In both cases,the Granger Causality Test indicates that when applying as few as 1 weekly lag and at most 2 weekly lags both the MOVE Index and the All Primary Dealer Treasury Volume datasets are useful in forecasting the FI Trading volume data.

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
data = weekly_df[['FI Trades','MOVE Index']]
data1 = weekly_df[['MOVE Index','FI Trades']]
data2 =weekly_df[['FI Trades','All Treasury Volume']]
data3 =weekly_df[['All Treasury Volume','FI Trades']]

print("*MOVE Index comparison*")
grangercausalitytests(data,maxlag=3);
print('_______________________________________')
print("*Primary Dealer comparison*")
grangercausalitytests(data2,maxlag=3);


In [ ]:
# Plot predictions against known values
title = 'Weekly Fixed Income Trading Volumes'
ylabel='Volume in mlns'
xlabel=''

ax = test['FI Trades'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)

# df1[df1['holiday']==1].index
for x in test[test['MOVE Index']>80].index: 
    ax.axvline(x=x, color='k', alpha = 0.3);


In [ ]:
model = SARIMAX(train['FI Trades'],exog=train['MOVE Index'],order=(0,1,2),seasonal_order=(1,0,0,52),enforce_invertibility=True)
results = model.fit()
results.summary()

Modest improvement in the AIC.  Let's model that result.

In [ ]:
# Obtain predicted values
start=len(train)
end=len(train)+len(test)-1
exog_forecast = test[['MOVE Index']]  # requires two brackets to yield a shape of (35,1)
predictions = results.predict(start=start, end=end, exog=exog_forecast).rename('SARIMAX(0, 1, 2)x(1, 0, [], 52)	')

In [ ]:
# Plot predictions against known values
title = 'Weekly Fixed Income Trading Volumes'
ylabel='Volume in mlns'
xlabel=''

ax = test['FI Trades'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)

# df1[df1['holiday']==1].index
for x in test[test['MOVE Index']>80].index: 
    ax.axvline(x=x, color='k', alpha = 0.3);

We can see some improvement in the early months.  Let's review the error again.

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test['FI Trades'], predictions)
print(f'SARIMAX(0, 1, 2)x(1, 0, [], 52)  RMSE Error: {error:11.10}')

In [ ]:
test_mean = test['FI Trades'].mean()
err_val = error/test_mean
print(f'Mean of the test set: {test_mean:11.8}')
print(f'Our relative error: {err_val:11.5}')


A significant improvement in the relative error from 23% to 16% yet still a big enough error to keep testing on another series.

What about the All Treasury data?

In [ ]:
# Plot predictions against known values
title = 'Weekly Fixed Income Trading Volumes'
ylabel='Volume in mlns'
xlabel=''

ax = test['FI Trades'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)

# df1[df1['holiday']==1].index
for x in test[test['All Treasury Volume']>500000].index: 
    ax.axvline(x=x, color='k', alpha = 0.3);

In [ ]:
model = SARIMAX(train['FI Trades'],exog=train['All Treasury Volume'],order=(0,1,2),seasonal_order=(1,0,0,52),enforce_invertibility=True)
results = model.fit()
results.summary()

In [ ]:
# Obtain predicted values
start=len(train)
end=len(train)+len(test)-1
exog_forecast = test[['All Treasury Volume']]  # requires two brackets to yield a shape of (35,1)
predictions = results.predict(start=start, end=end, exog=exog_forecast).rename('SARIMAX(0, 1, 2)x(1, 0, [], 52)	')

In [ ]:
# Plot predictions against known values
title = 'Weekly Fixed Income Trading Volumes'
ylabel='Volume in mlns'
xlabel=''

ax = test['FI Trades'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)

# df1[df1['holiday']==1].index
for x in test[test['All Treasury Volume']>500000].index: 
    ax.axvline(x=x, color='k', alpha = 0.3);

This appears to be a significant improvement.  Let's review the errors.

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test['FI Trades'], predictions)
print(f'SARIMAX(0, 1, 2)x(1, 0, [], 52)  RMSE Error: {error:11.10}')

In [ ]:
test_mean = test['FI Trades'].mean()
err_val = error/test_mean
print(f'Mean of the test set: {test_mean:11.8}')
print(f'Our relative error: {err_val:11.5}')

That's a solid improvement over the MOVE Index.  Now our tracking error is reflects an 11% margin.  Let's retrain the model and forecast the next 6 months, or 24 weeks from July of 2020.


In [ ]:
model = SARIMAX(weekly_df['FI Trades'],exog=weekly_df['All Treasury Volume'],order=(0,1,2),seasonal_order=(1,0,0,52),enforce_invertibility=False)
results = model.fit()
exog_forecast = weekly_df[-nobs:][['All Treasury Volume']]
fcast = results.predict(len(weekly_df),len(weekly_df)+23,exog=exog_forecast).rename('SARIMAX(0,1,2)(1,0,0,52) Forecast')

In [ ]:
# Plot predictions against known values
title = 'Weekly Fixed Income Trading Volumes'
ylabel='Volume in mlns'
xlabel=''

ax = weekly_df['FI Trades'][-nobs:].plot(legend=True,figsize=(12,8),title=title)
fcast.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);


By adding in the exogenous data of the Primary Dealer activity, we improved our tracking by over 10 points and used that model to predict future trading activity through the remainder of the year and through the uncertain election period and turmoil.